In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os

<h3>Load and preprocess datasets</h3>

In [2]:
df_v_new = pd.read_spss('data/caspian-5.sav',convert_categoricals=True)
df_v = pd.read_spss('data/last-caspian-v.sav',convert_categoricals=True) 
df_iv = pd.read_spss('data/caspian4_.sav',convert_categoricals=True)


In [8]:
df_iv.shape

(13983, 411)

In [3]:
df_v_new.id2.nunique()

13983

In [54]:
a = sorted(df_iv['University'].unique(), key=str)
b = sorted(df_v_new['University'].unique(), key=str)

# Compare the sorted lists
with open('new_5.txt', "w") as file:
      for item, ite in zip(a, b):  # Iterate over both a and b at the same time
        file.write(f'{item}\t{ite}\n')


In [34]:
a= df_v_new.University.unique().sort_values()
b= df_v.universi.unique().sort_values()
with open('new_5.txt', "w") as file:
    for item in a: # Iterate over both a and b at the same time
        file.write(f'{item}\n')


In [15]:
df_iv_prev.columns

Index(['id2', 'university', 'region', 'cluster', 'cross', 'birth_ye',
       'sample_c', 'a_1', 'a_2', 'a_3',
       ...
       'difficul', 'feelingd', 'multiple', 'tstmvpa', 'tyg', 'lunchcon',
       'dinnerco', 'lunchski', 'dinnersk', 'agebinar'],
      dtype='object', length=506)

In [18]:
df_iv = pd.read_spss('data/caspian4_.sav',convert_categoricals=True)
df_iv_prev = pd.read_spss('data/last-caspian-IV(1).sav',convert_categoricals=True)
df_v = pd.read_spss('data/last-caspian-v.sav',convert_categoricals=True) 
df_v_new = pd.read_spss('data/caspian-5.sav',convert_categoricals=True)
def rename_features(df1, caspian_number):
    df2 = df1.copy()
    # Rename the features in the dataframe
    if caspian_number == 55:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'University': 'university','ap_9':'schoolType'}, inplace=True)
    if caspian_number == 5:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 4:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType'}, inplace=True)
        df2.drop('sex', axis=1, inplace=True)
        # print(df2.columns)
        df2.rename(columns={'sex2': 'sex', 'weight_1': 'weight', 'height_2': 'height', 'University': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 3:
        df2.rename(columns={'weighte': 'weight', 'heighte': 'height', 'province': 'university', 'area':'region','p9':'schoolType'}, inplace=True)
    elif caspian_number == 1:
        df2.columns = df2.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)
        

    return df2

df_iv = rename_features(df_iv, 4)
df_iv_prev = rename_features(df_iv_prev, 4)

df_v = rename_features(df_v, 5)
df_v_new = rename_features(df_v_new, 55)

# Apply transformations to 'df_iv' and 'df_v'
df_iv['sex'] = df_iv['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)
df_iv['schoolType'] = df_iv['schoolType'].apply(lambda x: 'Public School' if x == 'dolati' else 'Private School' if x == 'gheyre entef' else 'Unknown')
df_v['schoolType'] = df_v['schoolType'].apply(lambda x: 'Public School' if x == 'dolati' else 'Private School' if x == 'gheyre entefai' else 'Unknown')
df_v_new['schoolType'] = df_v_new['schoolType'].apply(lambda x: 'Public School' if x == 'dolati' else 'Private School' if x == 'gheyre entefai' else 'Unknown')
df_v_new['sex'] = df_v_new['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)

# Now define df_dict with the modified dataframes
df_dict = {'caspian_IV_prev':df_iv_prev,'caspian_IV': df_iv, 'caspian_V': df_v,'caspian_V_new':df_v_new}

def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["weight", "height", "sex", "age"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["height", "weight", "sex"])
        print(
            f"Number of records with NaN value in weight or height in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs


# Usage
processed_dfs = preprocess(df_dict)

# Define the mapping dictionary
replacement_map = {
    '1 or 2 in we': '1 or 2 in week',
    '3 or 4 in we': '3 or 4 in week',
    '5 or 6 in we': '5 or 6 in week',
    'never': 'never',
    'everyday': 'everyday',
    '': np.nan  # map empty strings to NaN
}
replacement_ap_3= {
    'diplome': 'diplome',
    'bachelor': 'Bachelor',
    'primary': 'primary',
    'upper than b': 'upper than bachelor',
    '': np.nan,  # Empty string replaced by NaN
    'intermediate': 'intermediate',
    'illiterate': 'illiterate',
    'father died': 'Father died',
    'quranic lite': 'Quranic Literacy',
    'upper than bachelor':'upper than bachelor',
    'quranic literacy':'Quranic Literacy',
    'mother died' :'mother died'
}


# Apply the mapping to 'a_4' column, using the default value of an empty string
processed_dfs['caspian_IV']['a_4'] = processed_dfs['caspian_IV']['a_4'].apply(lambda x: replacement_map.get(x, ''))
processed_dfs['caspian_IV']['ap_3'] = processed_dfs['caspian_IV']['ap_3'].apply(lambda x: replacement_ap_3.get(x, ''))
processed_dfs['caspian_IV']['ap_4'] = processed_dfs['caspian_IV']['ap_4'].apply(lambda x: replacement_ap_3.get(x, ''))
processed_dfs['caspian_IV_prev']['ap_3'] = processed_dfs['caspian_IV_prev']['ap_3'].apply(lambda x: replacement_ap_3.get(x, ''))
processed_dfs['caspian_IV_prev']['ap_4'] = processed_dfs['caspian_IV_prev']['ap_4'].apply(lambda x: replacement_ap_3.get(x, ''))
processed_dfs['caspian_IV'].replace('', np.nan, inplace=True)
processed_dfs['caspian_V'].replace('', np.nan, inplace=True)

def replacement_function(df_dict, mappings):
    # Loop over each dataset in the dictionary
    for name, df in df_dict.items():
        # Get the appropriate mapping for each Caspian dataset
        university_to_province = mappings.get(name)
        if university_to_province:
    # Iterate over each DataFrame in df_dict
            for name, df in df_dict.items():
                # Replace values based on the mapping
                df_dict[name] = df.copy()  # Work with a copy if needed
                df_dict[name]['university'] = df_dict[name]['university'].replace(university_to_province)
                
                # Ensure the 'university' column is of type string
                df_dict[name]['university'] = df_dict[name]['university'].astype(str)
                
                # Sort the DataFrame by the 'university' column
                df_dict[name] = df_dict[name].sort_values(by='university', ascending=True)
        
        return df_dict

    # return df_dict

# # Define your mappings dictionary for each Caspian dataset
mappings = {
    'caspian_I': {
        "Gorgan": "Golestan",
        "Mashad": "Razavi Khorasan",
        "ShahidBeheshti": "Tehran",
        "Tabriz": "East Azerbaijan",
        "Yazd": "Yazd",
        "Rasht": "Gilan"
    },
    'caspian_III': {
        "4mahal bakhtyari": "Chaharmahal and Bakhtiari",
        "ardebil": "Ardabil",
        "azar gharbi": "West Azerbaijan",
        "boshehr": "Bushehr",
        "esfahan": "Isfahan",
        "fars": "Fars",
        "gazvin": "Qazvin",
        "gilan": "Gilan",
        "golestan": "Golestan",
        "gom": "Qom",
        "hamedan": "Hamedan",
        "hormozgan": "Hormozgan",
        "ilam": "Ilam",
        "kerman": "Kerman",
        "kermanshah": "Kermanshah",
        "khorasan jonobi": "South Khorasan",
        "khorasan razavi": "Razavi Khorasan",
        "khorasan shomali": "North Khorasan",
        "khozestan": "Khuzestan",
        "kordestan": "Kurdistan",
        "lorestan": "Lorestan",
        "markazi": "Markazi",
        "mazandaran": "Mazandaran",
        "semnan": "Semnan",
        "tehran": "Tehran",
        "yazd": "Yazd",
        "zanjan": "Zanjan"
    },
    'caspian_IV': {
        'charmahal': 'Chaharmahal and Bakhtiari',
        'alborz': 'Alborz',
        'dezful': 'Khuzestan',
        'mazandaran': 'Mazandaran',
        'sabzevar': 'Razavi Khorasan',
        'azar gharbi': 'West Azerbaijan',
        'kurdestan': 'Kurdistan',
        'kerman.jirof': 'Kerman',
        'ardebil': 'Ardabil',
        'hamedan': 'Hamedan',
        'khorasan sho': 'South Khorasan',
        'ilam': 'Ilam',
        'khozestan': 'Khuzestan',
        'shoshtar': 'Khuzestan',
        'qazvin': 'Qazvin',
        'isfahan': 'Isfahan',
        'gilan': 'Gilan',
        'tabriz': 'East Azerbaijan',
        'arak': 'Markazi',
        'semnan': 'Semnan',
        'hormozgan': 'Hormozgan',
        'lorestan': 'Lorestan',
        'boshehr': 'Bushehr',
        'zanjan': 'Zanjan',
        'sistan': 'Sistan and Baluchestan',
        'kermanshah': 'Kermanshah',
        'fars': 'Fars',
        'torbat heida': 'Razavi Khorasan',
        'kerman': 'Kerman',
        'khorasan jon': 'North Khorasan',
        'yazd': 'Yazd',
        'golestan': 'Golestan',
        'behbahan': 'Khuzestan',
        'shahidbehesh': 'Tehran',
        'shahrood': 'Semnan',
        'khorasan raz': 'Razavi Khorasan',
        'saveh': 'Markazi',
        'iran': 'General Iran',
        'esfarayen': 'North Khorasan',
        'kohkiloyeh': 'Kohgiluyeh and Boyer-Ahmad',
        'abadan': 'Khuzestan',
        'rafsanjan': 'Kerman',
        'iranshahr': 'Sistan and Baluchestan',
        'babol': 'Mazandaran',
        'torbat jam': 'Razavi Khorasan',
        'zabol': 'Sistan and Baluchestan',
        'kashan': 'Isfahan',
        'jahrom': 'Fars',
        'neishabor': 'Razavi Khorasan',
        'bam': 'Kerman',
        'tehran': 'Tehran',
        'larestan': 'Fars',
        'fasad': 'Fars'
    },
    'caspian_IV_prev':{
        "abadan": "Khuzestan",
        "alborz": "Alborz",
        "arak": "Markazi",
        "ardebil": "Ardabil",
        "azar gharbi": "West Azerbaijan",
        "babol": "Mazandaran",
        "bam": "Kerman",
        "behbahan": "Khuzestan",
        "boshehr": "Bushehr",
        "charmahal": "Chaharmahal and Bakhtiari",
        "dezful": "Khuzestan",
        "esfarayen": "North Khorasan",
        "fars": "Fars",
        "fasad": "Fars",
        "gilan": "Gilan",
        "golestan": "Golestan",
        "hamedan": "Hamedan",
        "hormozgan": "Hormozgan",
        "ilam": "Ilam",
        "iran": "General Iran",  # Special case: general reference
        "iranshahr": "Sistan and Baluchestan",
        "isfahan": "Isfahan",
        "jahrom": "Fars",
        "kashan": "Isfahan",
        "kerman": "Kerman",
        "kerman.jiroft": "Kerman",
        "kermanshah": "Kermanshah",
        "khorasan jonobi": "South Khorasan",
        "khorasan razavi": "Razavi Khorasan",
        "khorasan shomali": "North Khorasan",
        "khozestan": "Khuzestan",
        "kohkiloyeh": "Kohgiluyeh and Boyer-Ahmad",
        "kurdestan": "Kurdistan",
        "larestan": "Fars",
        "lorestan": "Lorestan",
        "mazandaran": "Mazandaran",
        "neishabor": "Razavi Khorasan",
        "qazvin": "Qazvin",
        "rafsanjan": "Kerman",
        "sabzevar": "Razavi Khorasan",
        "saveh": "Markazi",
        "semnan": "Semnan",
        "shahidbeheshti": "Tehran",  # Assuming Shahid Beheshti University is in Tehran
        "shahrood": "Semnan",
        "shoshtar": "Khuzestan",
        "sistan": "Sistan and Baluchestan",
        "tabriz": "East Azerbaijan",
        "tehran": "Tehran",
        "torbat heidariyeh": "Razavi Khorasan",
        "torbat jam": "Razavi Khorasan",
        "yazd": "Yazd",
        "zabol": "Sistan and Baluchestan",
        "zanjan": "Zanjan"
    },
    'caspian_V': {
        'tabriz': 'East Azerbaijan',
        # None: 'Unknown',  # Assuming None corresponds to an unknown city
        'azar gharbi': 'West Azerbaijan',
        'ardebil': 'Ardabil',
        'isfahan': 'Isfahan',
        'kashan': 'Isfahan',
        'alborz': 'Alborz',
        'ilam': 'Ilam',
        'boshehr': 'Bushehr',
        'tehran': 'Tehran',
        'Iran': 'General Iran',  # Assuming this is a general reference
        'shahidbeheshti': 'Tehran',  # Assuming it's named after Shahid Beheshti University in Tehran
        'charmahal': 'Chaharmahal and Bakhtiari',
        'khorasan jonobi': 'South Khorasan',
        'torbat heidariyeh': 'Razavi Khorasan',
        'sabzevar': 'Razavi Khorasan',
        'khorasan razavi': 'Razavi Khorasan',
        'neishabor': 'North Khorasan',
        'torbat jam': 'Razavi Khorasan',
        'khorasan shomali': 'North Khorasan',
        'esfarayen': 'North Khorasan',
        'khozestan': 'Khuzestan',
        'Abadan': 'Khuzestan',
        'Behbahan': 'Khuzestan',
        'Dezful': 'Khuzestan',
        'Shoshtar': 'Khuzestan',
        'Zanjan': 'Zanjan',
        'Semnan': 'Semnan',
        'Shahrood': 'Semnan',
        'Zabol': 'Sistan and Baluchestan',
        'Sistan': 'Sistan and Baluchestan',
        'iranshahr': 'Sistan and Baluchestan',
        'jahrom': 'Fars',
        'fars': 'Fars',
        'fasad': 'Fars',
        'larestan': 'Fars',
        'Qazvin': 'Qazvin',
        'Kurdestan': 'Kurdistan',
        'Kerman.jiroft': 'Kerman',
        'Rafsanjan': 'Kerman',
        'Kerman': 'Kerman',
        'Bam': 'Kerman',
        'kermanshah': 'Kermanshah',
        'Kohkiloyeh': 'Kohgiluyeh and Boyer-Ahmad',
        'Golestan': 'Golestan',
        'Gilan': 'Gilan',
        'Lorestan': 'Lorestan',
        'Babol': 'Mazandaran',
        'Mazandaran': 'Mazandaran',
        'Arak': 'Markazi',
        'saveh': 'Markazi',
        'Hormozgan': 'Hormozgan',
        'Hamedan': 'Hamedan',
        'Yazd': 'Yazd'
    },
    'caspian_V_new':{
        'charmahal': 'Chaharmahal and Bakhtiari',
        'alborz': 'Alborz',
        'dezful': 'Khuzestan',
        'mazandaran': 'Mazandaran',
        'sabzevar': 'Razavi Khorasan',
        'azar gharbi': 'West Azerbaijan',
        'kurdestan': 'Kurdistan',
        'kerman.jiroft': 'Kerman',
        'ardebil': 'Ardabil',
        'hamedan': 'Hamedan',
        'khorasan shomali': 'South Khorasan',
        'ilam': 'Ilam',
        'khozestan': 'Khuzestan',
        'shoshtar': 'Khuzestan',
        'qazvin': 'Qazvin',
        'isfahan': 'Isfahan',
        'gilan': 'Gilan',
        'tabriz': 'East Azerbaijan',
        'arak': 'Markazi',
        'semnan': 'Semnan',
        'hormozgan': 'Hormozgan',
        'lorestan': 'Lorestan',
        'boshehr': 'Bushehr',
        'zanjan': 'Zanjan',
        'sistan': 'Sistan and Baluchestan',
        'kermanshah': 'Kermanshah',
        'fars': 'Fars',
        'torbat heidariyeh': 'Razavi Khorasan',
        'kerman': 'Kerman',
        'khorasan jonobi': 'North Khorasan',
        'yazd': 'Yazd',
        'golestan': 'Golestan',
        'behbahan': 'Khuzestan',
        'shahidbeheshti': 'Tehran',
        'shahrood': 'Semnan',
        'khorasan razavi': 'Razavi Khorasan',
        'saveh': 'Markazi',
        'iran': 'General Iran',
        'esfarayen': 'North Khorasan',
        'kohkiloyeh': 'Kohgiluyeh and Boyer-Ahmad',
        'abadan': 'Khuzestan',
        'rafsanjan': 'Kerman',
        'iranshahr': 'Sistan and Baluchestan',
        'babol': 'Mazandaran',
        'torbat jam': 'Razavi Khorasan',
        'zabol': 'Sistan and Baluchestan',
        'kashan': 'Isfahan',
        'jahrom': 'Fars',
        'neishabor': 'Razavi Khorasan',
        'bam': 'Kerman',
        'tehran': 'Tehran',
        'larestan': 'Fars',
        'fasad': 'Fars'
    }
}

# Apply the function to the entire dictionary
# processed_dfs = replacement_function(processed_dfs, mappings)
university_to_province = mappings.get('caspian_IV')
processed_dfs['caspian_IV']['university'] = processed_dfs['caspian_IV']['university'].replace(university_to_province)
                
# Ensure the 'university' column is of type string
processed_dfs['caspian_IV']['university'] = processed_dfs['caspian_IV']['university'].astype(str)

# Sort the DataFrame by the 'university' column
processed_dfs['caspian_IV'] = processed_dfs['caspian_IV'].sort_values(by='university', ascending=True)

university_to_province = mappings.get('caspian_V')
processed_dfs['caspian_V']['university'] = processed_dfs['caspian_V']['university'].replace(university_to_province)
                
# Ensure the 'university' column is of type string
processed_dfs['caspian_V']['university'] = processed_dfs['caspian_V']['university'].astype(str)

# Sort the DataFrame by the 'university' column
processed_dfs['caspian_V'] = processed_dfs['caspian_V'].sort_values(by='university', ascending=True)

university_to_province = mappings.get('caspian_IV_prev')
processed_dfs['caspian_IV_prev']['university'] = processed_dfs['caspian_IV_prev']['university'].replace(university_to_province)
           
# Ensure the 'university' column is of type string
processed_dfs['caspian_IV_prev']['university'] = processed_dfs['caspian_IV_prev']['university'].astype(str)

# Sort the DataFrame by the 'university' column
processed_dfs['caspian_IV_prev'] = processed_dfs['caspian_IV_prev'].sort_values(by='university', ascending=True)

university_to_province = mappings.get('caspian_V_new')
processed_dfs['caspian_V_new']['university'] = processed_dfs['caspian_V_new']['university'].replace(university_to_province)
           
# Ensure the 'university' column is of type string
processed_dfs['caspian_V_new']['university'] = processed_dfs['caspian_V_new']['university'].astype(str)

# Sort the DataFrame by the 'university' column
processed_dfs['caspian_V_new'] = processed_dfs['caspian_V_new'].sort_values(by='university', ascending=True)

Number of records with NaN value in weight or height in caspian_IV_prev: 138
Number of records with NaN value in weight or height in caspian_IV: 143
Number of records with NaN value in weight or height in caspian_V: 128
Number of records with NaN value in weight or height in caspian_V_new: 143


/tmp/ipykernel_19397/1103639069.py:402: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  processed_dfs['caspian_V']['university'] = processed_dfs['caspian_V']['university'].replace(university_to_province)
/tmp/ipykernel_19397/1103639069.py:411: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  processed_dfs['caspian_IV_prev']['university'] = processed_dfs['caspian_IV_prev']['university'].replace(university_to_province)
/tmp/ipykernel_19397/1103639069.py:420: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a fu

<h3>Merging two dataset based on matching_features for finding similar records between two dataset</h3>

In [56]:
df1=processed_dfs['caspian_IV_prev']
df2=processed_dfs['caspian_V']

In [57]:
df1['ap_3'] = df1['ap_3'].str.lower()
df1['ap_4'] = df1['ap_4'].str.lower()


In [58]:
# List of features to compare
features = ['university','a_1', 'a_2', 'a_3', 'a_4', 'ap_2', 'ap_3', 'ap_4']  # Add your feature names here

# Dictionary to store results
comparison_results = {}

for feature in features:
    # Sort and convert unique values of the feature in each dataframe
    df1_unique_sorted = sorted(df1[feature].unique(), key=str)
    df2_unique_sorted = sorted(df2[feature].unique(), key=str)
    
    # Convert to sets
    df1_unique_set = set(df1_unique_sorted)
    df2_unique_set = set(df2_unique_sorted)
    
    # Find differences
    only_in_df1 = df1_unique_set - df2_unique_set
    only_in_df2 = df2_unique_set - df1_unique_set
    
    # Check equality and store results
    are_equal = not only_in_df1 and not only_in_df2
    comparison_results[feature] = {
        "are_equal": are_equal,
        "only_in_df1": only_in_df1,
        "only_in_df2": only_in_df2
    }

# Print comparison results
comparison_results


{'university': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'a_1': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'a_2': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'a_3': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'a_4': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'ap_2': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'ap_3': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()},
 'ap_4': {'are_equal': True, 'only_in_df1': set(), 'only_in_df2': set()}}

In [59]:

#  feature columns to match on
matching_features = ['cluster','sample_c','age', 'weight', 'height', 'university', 'region', 'wrist4', 'waist_3', 'systolic', 
                     'a_1', 'a_2', 'a_3', 'a_4', 'ap_2', 'ap_3', 'ap_4', 'familynu', 'birth_ye']
# Merge the datasets on matching features
merged_df = pd.merge(df1, df2, on=matching_features, suffixes=('_df1', '_df2'))

# Group by 'id2' from df2 and collect 'id2' values from df1
result = merged_df.groupby('id2_df2')['id2_df1'].apply(list).reset_index()

# Rename columns for clarity
result.columns = ['record_id2_df2', 'records_id2_df1']


# Step 1: Flatten the list of `id2` values from `records_id2_df1`
id2_to_remove = set([item for sublist in result['records_id2_df1'] for item in sublist])

# Step 2: Filter `caspian_IV` to exclude these `id2` values
df_filtered = df1[~df1['id2'].isin(id2_to_remove)]

# Display the filtered DataFrame
print(df_filtered.info())





<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 12333 to 2741
Columns: 507 entries, id2 to bmi1
dtypes: category(333), float64(170), object(4)
memory usage: 124.5+ KB
None


<h3>save intersection ids between 2 datasets</h3>

In [60]:
output_file = 'intersection_data_4_prev&5.xlsx'
with pd.ExcelWriter(output_file) as writer:
    result.to_excel(writer, index=False, sheet_name='ID2 Intersections')

<h3>Save new dataset that similar records deleted from it</h3>

In [54]:
import pyreadstat
# Save df_iv to a new SPSS file after modifications
pyreadstat.write_sav(df_filtered, 'data/caspian4_modified.sav')

<h3>Save new dataset that similar records deleted from it with matching feature column</h3>

In [55]:

import pyreadstat

matching_features = ['id2','cluster','sample_c','age', 'weight', 'height', 'university', 'region', 'wrist4', 'waist_3', 'systolic', 
                     'a_1', 'a_2', 'a_3', 'a_4', 'ap_2', 'ap_3', 'ap_4', 'familynu', 'birth_ye']
# Save df_iv to a new SPSS file after modifications
pyreadstat.write_sav(df_filtered[matching_features], 'data/selected_col/modified-caspian4_selected-col.sav')
pyreadstat.write_sav(df2[matching_features], 'data/selected_col/caspian5_selected-col.sav')


